# 06-14

- KNN Clustering 
- Using Categories from KNN to get subset from information
- Continous Features(Transition and Highlevel features)
- Categorical Features
- MinMax Scaling to all continous and categorical
- Remove Constant Features continous and categorical
- Gower

In [27]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, dropFeaturesWithNoVariance,loadCoverGroups
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy import stats
import dask_ml.cluster
import pandas as pd
import gower
from ipynb.fs.full.Pipeline import removeConstantFeatures, applyBoxCoxAllFeatures, centerAndScale, applyEuclideanDistance, getIndexAndVector, applyBoxCoxFeatures,applyJaccard, getEuclideanJaccardDf , applyCosineDistance, applyGower

In [28]:
cover = loadCoverGroups()
listCovers = cover[20]
originalSongId = listCovers[0]
coverSongId = listCovers[1]

In [29]:
transitionMatrices = pd.read_csv('./data/unified/07-transitionsClustered.csv')
transitionMatrices = transitionMatrices[['id','category']]

In [30]:
df = pd.read_csv('./data/unified/05-allDataContinousCategoricalTransition.csv')
ids = df.id
df = df.iloc[:,2:]

In [31]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,23,0.494,0.756,0.0432,0.001400,0.1640,0.589,133.337,0.000000,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,105,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,118,0.614,0.755,0.0276,0.016200,0.0831,0.645,92.622,0.000000,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120,0.319,0.807,0.0388,0.001040,0.1310,0.601,118.097,0.029412,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,292,0.496,0.527,0.0339,0.195000,0.1380,0.283,155.932,0.013889,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5972,6002496,0.509,0.537,0.0384,0.347000,0.0805,0.311,170.030,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5973,6002497,0.725,0.583,0.0427,0.300000,0.0665,0.762,79.984,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5974,6002498,0.399,0.787,0.0499,0.019700,0.0685,0.572,117.089,0.000000,0.015385,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5975,6002499,0.376,0.435,0.0264,0.513000,0.1380,0.304,80.002,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [32]:
df = pd.merge(df,transitionMatrices, on='id')

In [33]:
categoryOriginalSong = df[df.id == originalSongId].category.values[0]
df = df[df.category == categoryOriginalSong]
df = df.drop(columns=['category'])

In [34]:
## Removing unamed and other unnecessary columns
df.reset_index(drop=True, inplace=True)
df = df.sort_values('id')
df.id = df.id.astype('int64')
# Saving idChords
ids = df.id
df = df.set_index('id')
df.reset_index(drop=True, inplace=True)
df

,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,2,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,0.614,0.755,0.0276,0.01620,0.0831,0.6450,92.622,0.000000,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.496,0.527,0.0339,0.19500,0.1380,0.2830,155.932,0.013889,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.519,0.812,0.0278,0.00174,0.1540,0.6490,141.872,0.038095,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.457,0.488,0.0301,0.32600,0.1540,0.2830,72.517,0.025641,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.533,0.512,0.0288,0.04750,0.0442,0.5530,75.984,0.028571,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,0.676,0.572,0.0405,0.04490,0.0979,0.0398,149.997,0.070423,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1378,0.726,0.837,0.0320,0.25700,0.3220,0.5590,120.040,0.010000,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1379,0.845,0.938,0.0502,0.00306,0.0896,0.7550,127.007,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1380,0.548,0.542,0.1390,0.27400,0.0967,0.6490,174.043,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [35]:
df = removeConstantFeatures(df)
#df= applyBoxCoxFeatures(df, ['speechiness', 'acousticness', 'liveness', 'valence'])
#df= applyBoxCoxAllFeatures(df)
#df = centerAndScale(df)

In [36]:
scaler = MinMaxScaler()
scaler.fit(df)
df = pd.DataFrame(scaler.transform(df)).copy()

In [37]:
ranksDFGower = applyGower(df,ids, originalSongId,len(df))
ranksDFGower.head(10)

,id,values
0,6000771,0.000000
1,924061,0.013700
2,40013,0.015760
3,6000670,0.016257
4,118655,0.017571
5,2628012,0.017824
6,1709964,0.018146
7,582263,0.018240
8,695153,0.018393
9,1243779,0.018433


In [38]:
ranksDFGower.loc[ranksDFGower.id.isin(listCovers)]

,id,values
0,6000771,0.000000
3,6000670,0.016257


In [39]:
ranksDFGower[ranksDFGower['id']==coverSongId]

,id,values
3,6000670,0.016257
